Install unsloth using a GPU

In [ ]:
%%capture

!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,

)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Data Preprocessing for White Suprimacist dataset

In [ ]:
import pandas as pd
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Load data from JSON files
with open('./annotated_data-train.json', 'r') as f:
    train_data = json.load(f)

with open('./annotated_data-test.json', 'r') as f:
    test_data = json.load(f)


train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)


df = pd.concat([train_df, test_df])

data = df[['text', 'label']]


# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Split data into train and validation sets
X = list(data["text"])
y = list(data["label"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, stratify=y)


from datasets import Dataset, load_dataset
train_dataset = Dataset.from_dict({"text": X_train, "label": y_train})
val_dataset = Dataset.from_dict({"text": X_val, "label": y_val})

# Mapping labels to numerical values
label_map = {'hate': 1, 'noHate': 0}

# Apply mapping to train and validation labels
train_dataset = train_dataset.map(lambda example: {'label': label_map[example['label']]})
val_dataset = val_dataset.map(lambda example: {'label': label_map[example['label']]})

In [ ]:
print(list(set(train_dataset['label'])))

Creating Prompts

In [ ]:
from transformers import PreTrainedTokenizerFast

prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Output:
{}"""

EOS_TOKEN = tokenizer.eos_token

texts=[]
def formatting_prompts_func(examples):
    instruction = "You are an expert in identifying whether a statement contains sentiments against certain racial groups or ethnic communities. Given a statement, your task is to classify the given text to determine if it holds sentiments against racial groups or ethnic communities."
    input = examples["text"]
    #print("input: ",input)
    output = examples["label"]
    #print("output:",output)

    s=prompt.format(instruction, input, output)+' '+EOS_TOKEN
    #print(s)
    texts.append(s)
    return { "text" : s }


from datasets import load_dataset


dataset=train_dataset
dataset = dataset.map(formatting_prompts_func)
print(dataset)



Training Model on N-pair loss

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the NpairLoss class
class NpairLoss(nn.Module):
    def __init__(self, l2_reg=0.02):
        super(NpairLoss, self).__init__()
        self.l2_reg = l2_reg

    def forward(self, anchor, positive, target):
        batch_size = anchor.size(0)
        target = target.view(target.size(0), 1)

        target = (target == torch.transpose(target, 0, 1)).float()
        target = target / torch.sum(target, dim=1, keepdim=True).float()

        logit = torch.matmul(anchor, torch.transpose(positive, 0, 1))
        loss_ce = F.cross_entropy(logit, target)
        l2_loss = torch.sum(anchor**2) / batch_size + torch.sum(positive**2) / batch_size

        loss = loss_ce + self.l2_reg*l2_loss*0.25
        return loss

# Create an instance of the NpairLoss
npair_loss = NpairLoss()

# Instantiate the trainer without specifying compute_loss argument
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    )
)

# Set the custom loss function to the model
trainer.model.compute_loss = npair_loss
trainer_stats = trainer.train()


Inference/Testing.

In [ ]:
from transformers import PreTrainedTokenizerFast

prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Output:
"""

texts=[]
def formatting_prompts_func(examples):
    instruction = "You are an expert in identifying whether a statement contains sentiments against certain racial groups or ethnic communities. Given a statement, your task is to classify the given text to determine if it holds sentiments against racial groups or ethnic communities."
    input = examples["text"]
    #print("input: ",input)
    output = examples["label"]
    #print("output:",output)

    s=prompt.format(instruction, input)
    #print(s)
    texts.append(s)
    return { "text" : s }


from datasets import load_dataset

test_dataset=val_dataset.map(formatting_prompts_func)


In [ ]:
%%capture
import time

generated_outputs = []
FastLanguageModel.for_inference(model)

# Iterate through the test dataset
for idx in range(len(test_dataset['text'])):
    # Generate input prompt
    prompt_input = tokenizer(test_dataset['text'][idx],return_tensors="pt").to("cuda")

    # Generate output
    outputs = model.generate(**prompt_input, max_new_tokens = 64, use_cache = True).to("cuda")

    print(tokenizer.batch_decode(outputs))
    # Append generated output to the list
    generated_outputs.append(tokenizer.batch_decode(outputs))

In [ ]:
generated_labels = [item[0].split("### Output:\n")[1].strip().split(" ")[0] for item in generated_outputs]

print(list(set(generated_labels)))

Evaluating performance

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Convert generated_outputs to binary labels (0 or 1)
predicted_labels = [1 if label == "hate" else 0 for label in generated_outputs]

# Convert y_val to binary labels (0 or 1)
actual_labels = [1 if label == "hate" else 0 for label in y_val]

# Calculate accuracy
accuracy = accuracy_score(actual_labels, predicted_labels)

# Calculate precision
precision = precision_score(actual_labels, predicted_labels)

# Calculate recall
recall = recall_score(actual_labels, predicted_labels)

# Calculate F1 score
f1 = f1_score(actual_labels, predicted_labels)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Save current model

In [ ]:
model.save_pretrained("supremacist")
tokenizer.save_pretrained("supremacist")

In [ ]:
import shutil

# Zip the folder containing model and tokenizer files
shutil.make_archive("supremacist", "zip", "supremacist")